In [82]:
import pickle

import pandas as pd
from tqdm.notebook import tqdm
tqdm.pandas()

pd.set_option("display.max_colwidth", None)

In [83]:
examples = {}

for i in range(0, 5):
    with open("../../out/definitions.pickle", "rb") as definitions_file:
        with open(f"../../out/llama/examples_{i}_shot_v4.pickle", "rb") as examples_file:
            examples[i] = {}
            cur_definitions = pickle.load(definitions_file)
            cur_examples = pickle.load(examples_file)
            for (d, de), ex in zip(cur_definitions.items(), cur_examples):
                if i == 0:
                    examples[i][d] = (de, [], ex)
                else:
                    examples[i][d] = (de, *ex[:2])
    examples[i] = pd.DataFrame.from_dict(examples[i], orient="index", columns=["definition", "examples", "generated"])

Add levenshtein ratio:

In [84]:
from Levenshtein import ratio

def ex_gen_levenshtein(row):
    ex = row["examples"]
    try:
        gen = eval(row["generated"])
        lev = 0
        for e in ex:
            for g in gen:
               lev += ratio(str(e), str(g))
        if len(ex) * len(gen) > 0:
            lev /= len(ex) * len(gen)
        else:
            lev = float("NaN")
        return lev
    except SyntaxError:
        return float("NaN")
    
    
def def_gen_levenshtein(row):
    de = row["definition"]
    try:
        gen = eval(row["generated"])
        lev = 0
        for g in gen:
           lev += ratio(str(de), str(g))
        if len(gen) > 0:
            lev /= len(gen)
        else:
            lev = float("NaN")
        return lev
    except SyntaxError:
        return float("NaN")

for e in examples:
    examples[e][f"{e}_ex_gen_levenshtein"] = examples[e].progress_apply(ex_gen_levenshtein, axis=1)
    examples[e][f"{e}_def_gen_levenshtein"] = examples[e].progress_apply(def_gen_levenshtein, axis=1)

  0%|          | 0/211 [00:00<?, ?it/s]

  0%|          | 0/211 [00:00<?, ?it/s]

  0%|          | 0/211 [00:00<?, ?it/s]

  0%|          | 0/211 [00:00<?, ?it/s]

  0%|          | 0/211 [00:00<?, ?it/s]

  0%|          | 0/211 [00:00<?, ?it/s]

  0%|          | 0/86 [00:00<?, ?it/s]

  0%|          | 0/86 [00:00<?, ?it/s]

  0%|          | 0/211 [00:00<?, ?it/s]

  0%|          | 0/211 [00:00<?, ?it/s]

Jaro-Winkler score:

In [85]:
from Levenshtein import jaro_winkler

def ex_gen_jaro_winkler(row):
    ex = row["examples"]
    try:
        gen = eval(row["generated"])
        lev = 0
        for e in ex:
            for g in gen:
               lev += jaro_winkler(str(e), str(g))
        if len(ex) * len(gen) > 0:
            lev /= len(ex) * len(gen)
        else:
            lev = float("NaN")
        return lev
    except SyntaxError:
        return float("NaN")
    
    
def def_gen_jaro_winkler(row):
    de = row["definition"]
    try:
        gen = eval(row["generated"])
        lev = 0
        for g in gen:
           lev += jaro_winkler(str(de), str(g))
        if len(gen) > 0:
            lev /= len(gen)
        else:
            lev = float("NaN")
        return lev
    except SyntaxError:
        return float("NaN")

for e in examples:
    examples[e][f"{e}_ex_gen_jaro_winkler"] = examples[e].progress_apply(ex_gen_jaro_winkler, axis=1)
    examples[e][f"{e}_def_gen_jaro_winkler"] = examples[e].progress_apply(def_gen_jaro_winkler, axis=1)

  0%|          | 0/211 [00:00<?, ?it/s]

  0%|          | 0/211 [00:00<?, ?it/s]

  0%|          | 0/211 [00:00<?, ?it/s]

  0%|          | 0/211 [00:00<?, ?it/s]

  0%|          | 0/211 [00:00<?, ?it/s]

  0%|          | 0/211 [00:00<?, ?it/s]

  0%|          | 0/86 [00:00<?, ?it/s]

  0%|          | 0/86 [00:00<?, ?it/s]

  0%|          | 0/211 [00:00<?, ?it/s]

  0%|          | 0/211 [00:00<?, ?it/s]

BLEU score:

In [86]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Tim\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [87]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.tokenize import word_tokenize, sent_tokenize

def ex_gen_bleu(row):
    ex = [word_tokenize(e) for e in row["examples"]]
    gen = [word_tokenize(g) for g in sent_tokenize(row["generated"][1:-1])]
    bleu = 0
    smoothing = SmoothingFunction()
    try:
        for g in gen:
           bleu += sentence_bleu(ex, str(g), smoothing_function=smoothing.method2)
        if len(gen) > 0:
            bleu /= len(gen)
        else:
            bleu = float("NaN")
        return bleu
    except:
        return float("NaN")
    
    
def def_gen_bleu(row):
    de = [word_tokenize(d) for d in sent_tokenize(row["definition"])]
    gen = [word_tokenize(g) for g in sent_tokenize(row["generated"][1:-1])]
    bleu = 0
    smoothing = SmoothingFunction()
    try:
        for g in gen:
           bleu += sentence_bleu(de, str(g), smoothing_function=smoothing.method2)
        if len(gen) > 0:
            bleu /= len(gen)
        else:
            bleu = float("NaN")
        return bleu
    except:
        return float("NaN")

for e in examples:
    examples[e][f"{e}_ex_gen_bleu"] = examples[e].progress_apply(ex_gen_bleu, axis=1)
    examples[e][f"{e}_def_gen_bleu"] = examples[e].progress_apply(def_gen_bleu, axis=1)

  0%|          | 0/211 [00:00<?, ?it/s]

  0%|          | 0/211 [00:00<?, ?it/s]

  0%|          | 0/211 [00:00<?, ?it/s]

  0%|          | 0/211 [00:00<?, ?it/s]

  0%|          | 0/211 [00:00<?, ?it/s]

  0%|          | 0/211 [00:00<?, ?it/s]

  0%|          | 0/86 [00:00<?, ?it/s]

  0%|          | 0/86 [00:00<?, ?it/s]

  0%|          | 0/211 [00:00<?, ?it/s]

  0%|          | 0/211 [00:00<?, ?it/s]

In [88]:
print("Levenshtein ratio:")
print(examples[0]["0_ex_gen_levenshtein"].mean())
print(examples[0]["0_ex_gen_levenshtein"].std())
print("-----")
print(examples[0]["0_def_gen_levenshtein"].mean())
print(examples[0]["0_def_gen_levenshtein"].std())
print("=====")
print("Jaro-Winkler similarity score:")
print(examples[0]["0_ex_gen_jaro_winkler"].mean())
print(examples[0]["0_ex_gen_jaro_winkler"].std())
print("-----")
print(examples[0]["0_def_gen_jaro_winkler"].mean())
print(examples[0]["0_def_gen_jaro_winkler"].std())
print("=====")
print("BLEU score:")
print(examples[0]["0_ex_gen_bleu"].mean())
print(examples[0]["0_ex_gen_bleu"].std())
print("-----")
print(examples[0]["0_def_gen_bleu"].mean())
print(examples[0]["0_def_gen_bleu"].std())
examples[0]

Levenshtein ratio:
nan
nan
-----
0.10114475565618114
0.0620259342675694
=====
Jaro-Winkler similarity score:
nan
nan
-----
0.5292155267564049
0.03532567550015861
=====
BLEU score:
nan
nan
-----
0.01074641159259956
0.00845470715247841


,definition,examples,generated,0_ex_gen_levenshtein,0_def_gen_levenshtein,0_ex_gen_jaro_winkler,0_def_gen_jaro_winkler,0_ex_gen_bleu,0_def_gen_bleu
10,"Die ""Negation:NEG_XgeschweigedennY-Konstruktion"" korreliert zwei Propositionen, die jeweils einen Punkt auf einem pragmatisch definiertem Maßstab markieren. Die erste Proposition ist pragmatisch stärker als die zweite Proposition; so zieht, pragmatisch betrachtet, die Wahrheit der ersten Proposition automatisch die Wahrheit der zweiten Proposition nach sich. Einfacher ausgedrückt: Ist die erste Proposition wahr, so muss die zweite Proposition ebenfalls wahr sein. Die erste Proposition ist darüber hinaus auch informativer als die zweite Proposition, da die erstgenannte pragmatisch betrachtet die letztgenannte einschließt, während diese wiederum die diskursrelevantere der beiden Propositionen ist. Die Konstruktion umfasst das konstruktionsevozierende Element (KEE) ""geschweige_denn"", die internen Kern-Konstruktionselemente (Kern-KE) ""Erstes_Konjunkt"" und ""Zweites_Konjunkt"" sowie die externen Kern-KE ""Negator"" und ""Fokuskontext"". Das ""Erste_Konjunkt"" geht dem ""KEE"" voraus, auf welches das ""Zweite_Konjunkt"" folgt. Der informationsstrukturelle Fokus liegt auf dem ""Ersten_Konjunkt"", das die Basis für den entstehenden Kontrast zwischen den zwei Propositionen bildet. Das ""Erste_Konjunkt"" und das ""Zweite_Konjunkt"" bilden jeweils einen Teil der beiden Propositionen ab, wobei das ""Erste_Konjunkt"" meist zusätzlih vom ""Negator"" negiert wird. Der ""Fokuskontext"" indiziert in der Regel den Rest der Proposition. Werden beide Konjunkte im Wechsel mit dem ""Fokuskontext"" kombiniert, können die einander gegenübergestellten Propositionen vollständig wiederhergestellt (oder: vervollständigt) werden. Im Gegensatz zum ""Ersten_Konjunkt"" und ""Zweiten_Konjunkt"" muss der ""Fokuskontext"" nicht zwangsläufig realisiert werden – wird der ""Fokuskontext"" nicht realisiert, drücken die beiden Konjunkte jeweils vollständige Propositionen aus. "" Die zwei gegenübergestellten Propositionen sind hier die beiden Aussagen, dass die meisten von ihnen zuvor noch nie einen Computer a) gesehen und b) bedient haben. Die Verwendung der ""Negation:NEG_XgeschweigedennY-Konstruktion"" erfordert die Einstufung von Sehen und Bedienen auf einer 'pragmatischen Skala', was dazu führt, dass die Aussage Die meisten von ihnen haben zuvor noch nie einen Computer bedient als 'größere' Behauptung betrachtet werden muss als Die meisten von ihnen haben zuvor noch nie einen Computer gesehen. Mit anderen Worten: Wenn die meisten von ihnen einen Computer zuvor noch nie gesehen haben, dann haben sie ihn ganz sicher auch nicht bedient bzw. wenn sie einen Computer noch nie gesehen haben, können sie diesen auch nicht bedient haben. Proposition a) schließt Proposition b) somit aus. Unter dem Aspekt der Negation wird die erste Proposition Die meisten von ihnen haben zuvor noch nie einen Computer gesehen damit zur stärkeren der beiden kontrastierten Propositionen.",[],"[""Die meisten von ihnen haben zuvor noch nie einen Computer gesehen."", ""Die meisten von ihnen haben zuvor noch nie einen Computer bedient.""]",NaN,0.045486,NaN,0.533359,NaN,0.016514
100,"Die ""Äquativ_Plural-Konstruktion"" gehört zu den Vergleichskonstruktionen und zeigt die Gleichheit zweier Entitäten an, die hinsichtlich eines bestimmten Wertes eines spezifischen Attributs auf einer Skala miteinander verglichen werden. Der Vergleichsausdruck besteht aus dem konstruktionsevozierenden Element (KEE) ""gleich"", das angibt, um welche Art des Vergleichs es sich handelt (hier: Gleichheit und nicht Ungleichheit), und den beiden Kern-Konstruktionselementen (KE) ""Verglichene_Entitäten"" und ""Dimension"". Dabei umfasst das KE ""Verglichene_Entitäten"" diejenigen Einheiten, die hinsichtlich eines bestimmten Wertes einer durch die Vergleichskategorie definierten Skala gleichgesetzt werden. Die Skala wird dabei durch die ""Dimension"" angezeigt (X und Y s

In [89]:
print("Levenshtein ratio:")
print(examples[1]["1_ex_gen_levenshtein"].mean())
print(examples[1]["1_ex_gen_levenshtein"].std())
print("-----")
print(examples[1]["1_def_gen_levenshtein"].mean())
print(examples[1]["1_def_gen_levenshtein"].std())
print("=====")
print("Jaro-Winkler similarity score:")
print(examples[1]["1_ex_gen_jaro_winkler"].mean())
print(examples[1]["1_ex_gen_jaro_winkler"].std())
print("-----")
print(examples[1]["1_def_gen_jaro_winkler"].mean())
print(examples[1]["1_def_gen_jaro_winkler"].std())
print("=====")
print("BLEU score:")
print(examples[1]["1_ex_gen_bleu"].mean())
print(examples[1]["1_ex_gen_bleu"].std())
print("-----")
print(examples[1]["1_def_gen_bleu"].mean())
print(examples[1]["1_def_gen_bleu"].std())
examples[1]

Levenshtein ratio:
0.03202020538900645
0.013491980548376026
-----
0.13416783200991286
0.07466546867492732
=====
Jaro-Winkler similarity score:
0.45296166028232177
0.08916730540202354
-----
0.5304984578265755
0.03072053834165266
=====
BLEU score:
0.014727100861931732
0.010794883568769339
-----
0.008888787680779215
0.0072444791471314085


,definition,examples,generated,1_ex_gen_levenshtein,1_def_gen_levenshtein,1_ex_gen_jaro_winkler,1_def_gen_jaro_winkler,1_ex_gen_bleu,1_def_gen_bleu
10,"Die ""Negation:NEG_XgeschweigedennY-Konstruktion"" korreliert zwei Propositionen, die jeweils einen Punkt auf einem pragmatisch definiertem Maßstab markieren. Die erste Proposition ist pragmatisch stärker als die zweite Proposition; so zieht, pragmatisch betrachtet, die Wahrheit der ersten Proposition automatisch die Wahrheit der zweiten Proposition nach sich. Einfacher ausgedrückt: Ist die erste Proposition wahr, so muss die zweite Proposition ebenfalls wahr sein. Die erste Proposition ist darüber hinaus auch informativer als die zweite Proposition, da die erstgenannte pragmatisch betrachtet die letztgenannte einschließt, während diese wiederum die diskursrelevantere der beiden Propositionen ist. Die Konstruktion umfasst das konstruktionsevozierende Element (KEE) ""geschweige_denn"", die internen Kern-Konstruktionselemente (Kern-KE) ""Erstes_Konjunkt"" und ""Zweites_Konjunkt"" sowie die externen Kern-KE ""Negator"" und ""Fokuskontext"". Das ""Erste_Konjunkt"" geht dem ""KEE"" voraus, auf welches das ""Zweite_Konjunkt"" folgt. Der informationsstrukturelle Fokus liegt auf dem ""Ersten_Konjunkt"", das die Basis für den entstehenden Kontrast zwischen den zwei Propositionen bildet. Das ""Erste_Konjunkt"" und das ""Zweite_Konjunkt"" bilden jeweils einen Teil der beiden Propositionen ab, wobei das ""Erste_Konjunkt"" meist zusätzlih vom ""Negator"" negiert wird. Der ""Fokuskontext"" indiziert in der Regel den Rest der Proposition. Werden beide Konjunkte im Wechsel mit dem ""Fokuskontext"" kombiniert, können die einander gegenübergestellten Propositionen vollständig wiederhergestellt (oder: vervollständigt) werden. Im Gegensatz zum ""Ersten_Konjunkt"" und ""Zweiten_Konjunkt"" muss der ""Fokuskontext"" nicht zwangsläufig realisiert werden – wird der ""Fokuskontext"" nicht realisiert, drücken die beiden Konjunkte jeweils vollständige Propositionen aus. "" Die zwei gegenübergestellten Propositionen sind hier die beiden Aussagen, dass die meisten von ihnen zuvor noch nie einen Computer a) gesehen und b) bedient haben. Die Verwendung der ""Negation:NEG_XgeschweigedennY-Konstruktion"" erfordert die Einstufung von Sehen und Bedienen auf einer 'pragmatischen Skala', was dazu führt, dass die Aussage Die meisten von ihnen haben zuvor noch nie einen Computer bedient als 'größere' Behauptung betrachtet werden muss als Die meisten von ihnen haben zuvor noch nie einen Computer gesehen. Mit anderen Worten: Wenn die meisten von ihnen einen Computer zuvor noch nie gesehen haben, dann haben sie ihn ganz sicher auch nicht bedient bzw. wenn sie einen Computer noch nie gesehen haben, können sie diesen auch nicht bedient haben. Proposition a) schließt Proposition b) somit aus. Unter dem Aspekt der Negation wird die erste Proposition Die meisten von ihnen haben zuvor noch nie einen Computer gesehen damit zur stärkeren der beiden kontrastierten Propositionen.","['Es ist ein Foto , nur ein Foto , schwarz - weiß , darauf zu sehen eine stolze und gütige alte Frau , die Silberhaare straff nach hinten gesteckt , schwarz gekleidet , das Tuch geht bis zum Boden .']","[""Es ist ein schwarz-weißes Foto"", ""Es ist ein schwarzes Foto mit Silberhaar""]",0.044634,0.024027,0.446333,0.537068,0.016245,0.013661
100,"Die ""Äquativ_Plural-Konstruktion"" gehört zu den Vergleichskonstruktionen und zeigt die Gleichheit zweier Entitäten an, die hinsichtlich eines bestimmten Wertes eines spezifischen Attributs auf einer Skala miteinander verglichen werden. Der Vergleichsausdruck besteht aus dem konstruktionsevozierenden Element (KEE) ""gleich"", das angibt, um welche Art des Vergleichs es sich handelt (hier: Gleichheit und nicht Ungleichheit), und den beiden Kern-Konstruktionselementen (KE) ""Verglichene_Entitäten"" und ""Dimension"". Dabei umfasst das KE ""Verglichene_Entitäten"" diejenigen Einheiten, die hinsichtlich eines bestim

In [90]:
print("Levenshtein ratio:")
print(examples[2]["2_ex_gen_levenshtein"].mean())
print(examples[2]["2_ex_gen_levenshtein"].std())
print("-----")
print(examples[2]["2_def_gen_levenshtein"].mean())
print(examples[2]["2_def_gen_levenshtein"].std())
print("=====")
print("Jaro-Winkler similarity score:")
print(examples[2]["2_ex_gen_jaro_winkler"].mean())
print(examples[2]["2_ex_gen_jaro_winkler"].std())
print("-----")
print(examples[2]["2_def_gen_jaro_winkler"].mean())
print(examples[2]["2_def_gen_jaro_winkler"].std())
print("=====")
print("BLEU score:")
print(examples[2]["2_ex_gen_bleu"].mean())
print(examples[2]["2_ex_gen_bleu"].std())
print("-----")
print(examples[2]["2_def_gen_bleu"].mean())
print(examples[2]["2_def_gen_bleu"].std())
examples[2]

Levenshtein ratio:
0.031095286256492386
0.015107604348162322
-----
0.14181002538542156
0.07559574531738042
=====
Jaro-Winkler similarity score:
0.4626004951772992
0.08052875419973504
-----
0.5337409359706476
0.023204684842052167
=====
BLEU score:
0.015225751105127704
0.008972366771576337
-----
0.009336242785646594
0.006800328622785435


,definition,examples,generated,2_ex_gen_levenshtein,2_def_gen_levenshtein,2_ex_gen_jaro_winkler,2_def_gen_jaro_winkler,2_ex_gen_bleu,2_def_gen_bleu
10,"Die ""Negation:NEG_XgeschweigedennY-Konstruktion"" korreliert zwei Propositionen, die jeweils einen Punkt auf einem pragmatisch definiertem Maßstab markieren. Die erste Proposition ist pragmatisch stärker als die zweite Proposition; so zieht, pragmatisch betrachtet, die Wahrheit der ersten Proposition automatisch die Wahrheit der zweiten Proposition nach sich. Einfacher ausgedrückt: Ist die erste Proposition wahr, so muss die zweite Proposition ebenfalls wahr sein. Die erste Proposition ist darüber hinaus auch informativer als die zweite Proposition, da die erstgenannte pragmatisch betrachtet die letztgenannte einschließt, während diese wiederum die diskursrelevantere der beiden Propositionen ist. Die Konstruktion umfasst das konstruktionsevozierende Element (KEE) ""geschweige_denn"", die internen Kern-Konstruktionselemente (Kern-KE) ""Erstes_Konjunkt"" und ""Zweites_Konjunkt"" sowie die externen Kern-KE ""Negator"" und ""Fokuskontext"". Das ""Erste_Konjunkt"" geht dem ""KEE"" voraus, auf welches das ""Zweite_Konjunkt"" folgt. Der informationsstrukturelle Fokus liegt auf dem ""Ersten_Konjunkt"", das die Basis für den entstehenden Kontrast zwischen den zwei Propositionen bildet. Das ""Erste_Konjunkt"" und das ""Zweite_Konjunkt"" bilden jeweils einen Teil der beiden Propositionen ab, wobei das ""Erste_Konjunkt"" meist zusätzlih vom ""Negator"" negiert wird. Der ""Fokuskontext"" indiziert in der Regel den Rest der Proposition. Werden beide Konjunkte im Wechsel mit dem ""Fokuskontext"" kombiniert, können die einander gegenübergestellten Propositionen vollständig wiederhergestellt (oder: vervollständigt) werden. Im Gegensatz zum ""Ersten_Konjunkt"" und ""Zweiten_Konjunkt"" muss der ""Fokuskontext"" nicht zwangsläufig realisiert werden – wird der ""Fokuskontext"" nicht realisiert, drücken die beiden Konjunkte jeweils vollständige Propositionen aus. "" Die zwei gegenübergestellten Propositionen sind hier die beiden Aussagen, dass die meisten von ihnen zuvor noch nie einen Computer a) gesehen und b) bedient haben. Die Verwendung der ""Negation:NEG_XgeschweigedennY-Konstruktion"" erfordert die Einstufung von Sehen und Bedienen auf einer 'pragmatischen Skala', was dazu führt, dass die Aussage Die meisten von ihnen haben zuvor noch nie einen Computer bedient als 'größere' Behauptung betrachtet werden muss als Die meisten von ihnen haben zuvor noch nie einen Computer gesehen. Mit anderen Worten: Wenn die meisten von ihnen einen Computer zuvor noch nie gesehen haben, dann haben sie ihn ganz sicher auch nicht bedient bzw. wenn sie einen Computer noch nie gesehen haben, können sie diesen auch nicht bedient haben. Proposition a) schließt Proposition b) somit aus. Unter dem Aspekt der Negation wird die erste Proposition Die meisten von ihnen haben zuvor noch nie einen Computer gesehen damit zur stärkeren der beiden kontrastierten Propositionen.","['Es geht um das Bauprogramm des rot - grünen Senats .', 'Im Grunde war es Amerikas Konservativen gelungen , dem Gegner ihre eigene populistische Erzählung einzuflößen : dass die USA eben ein Mitte - rechts - Land seien und die abgehobenen Küsten - Eliten keine Ahnung hätten vom Leben und Denken des amerikanischenheartland .']","[""Die Schüler des künstenstudentischen Fachbereichs müssen sich ständig über ihre Prüfungsleistungen beschweren, aber der Kunstdekan sagt ihnen stets, dass sie es schon selbst machen können"", ""Erstens sagte der Lehrer nicht mal etwas, dann kam sie einfach zu spät, und dann war sie doch zu früh dran"", ""Das neue Musikvideo des Popstars war ganz in der Art wie man es von ihm erwartet hatte, genauso wie es in den Jahren zuvor immer wieder so war""]",0.012698,0.093497,0.560966,0.530770,0.003377,0.002566
100,"Die ""Äquativ_Plural-Konstruktion"" gehört zu den Vergleichskonstruktionen und zeigt die Gleichheit zweier Entitäten an, 

In [91]:
# TODO Has only 86 rows instead of 211
print("Levenshtein ratio:")
print(examples[3]["3_ex_gen_levenshtein"].mean())
print(examples[3]["3_ex_gen_levenshtein"].std())
print("-----")
print(examples[3]["3_def_gen_levenshtein"].mean())
print(examples[3]["3_def_gen_levenshtein"].std())
print("=====")
print("Jaro-Winkler similarity score:")
print(examples[3]["3_ex_gen_jaro_winkler"].mean())
print(examples[3]["3_ex_gen_jaro_winkler"].std())
print("-----")
print(examples[3]["3_def_gen_jaro_winkler"].mean())
print(examples[3]["3_def_gen_jaro_winkler"].std())
print("=====")
print("BLEU score:")
print(examples[3]["3_ex_gen_bleu"].mean())
print(examples[3]["3_ex_gen_bleu"].std())
print("-----")
print(examples[3]["3_def_gen_bleu"].std())
print(examples[3]["3_def_gen_bleu"].mean())
examples[3]

Levenshtein ratio:
0.034333218863796866
0.014902791283742334
-----
0.13613946281007325
0.07115746249762238
=====
Jaro-Winkler similarity score:
0.44450024925406706
0.08092171452502514
-----
0.533688222498089
0.0213815992721509
=====
BLEU score:
0.014347835961160064
0.009868158369067197
-----
0.006438493847598917
0.008554543335161037


,definition,examples,generated,3_ex_gen_levenshtein,3_def_gen_levenshtein,3_ex_gen_jaro_winkler,3_def_gen_jaro_winkler,3_ex_gen_bleu,3_def_gen_bleu
10,"Die ""Negation:NEG_XgeschweigedennY-Konstruktion"" korreliert zwei Propositionen, die jeweils einen Punkt auf einem pragmatisch definiertem Maßstab markieren. Die erste Proposition ist pragmatisch stärker als die zweite Proposition; so zieht, pragmatisch betrachtet, die Wahrheit der ersten Proposition automatisch die Wahrheit der zweiten Proposition nach sich. Einfacher ausgedrückt: Ist die erste Proposition wahr, so muss die zweite Proposition ebenfalls wahr sein. Die erste Proposition ist darüber hinaus auch informativer als die zweite Proposition, da die erstgenannte pragmatisch betrachtet die letztgenannte einschließt, während diese wiederum die diskursrelevantere der beiden Propositionen ist. Die Konstruktion umfasst das konstruktionsevozierende Element (KEE) ""geschweige_denn"", die internen Kern-Konstruktionselemente (Kern-KE) ""Erstes_Konjunkt"" und ""Zweites_Konjunkt"" sowie die externen Kern-KE ""Negator"" und ""Fokuskontext"". Das ""Erste_Konjunkt"" geht dem ""KEE"" voraus, auf welches das ""Zweite_Konjunkt"" folgt. Der informationsstrukturelle Fokus liegt auf dem ""Ersten_Konjunkt"", das die Basis für den entstehenden Kontrast zwischen den zwei Propositionen bildet. Das ""Erste_Konjunkt"" und das ""Zweite_Konjunkt"" bilden jeweils einen Teil der beiden Propositionen ab, wobei das ""Erste_Konjunkt"" meist zusätzlih vom ""Negator"" negiert wird. Der ""Fokuskontext"" indiziert in der Regel den Rest der Proposition. Werden beide Konjunkte im Wechsel mit dem ""Fokuskontext"" kombiniert, können die einander gegenübergestellten Propositionen vollständig wiederhergestellt (oder: vervollständigt) werden. Im Gegensatz zum ""Ersten_Konjunkt"" und ""Zweiten_Konjunkt"" muss der ""Fokuskontext"" nicht zwangsläufig realisiert werden – wird der ""Fokuskontext"" nicht realisiert, drücken die beiden Konjunkte jeweils vollständige Propositionen aus. "" Die zwei gegenübergestellten Propositionen sind hier die beiden Aussagen, dass die meisten von ihnen zuvor noch nie einen Computer a) gesehen und b) bedient haben. Die Verwendung der ""Negation:NEG_XgeschweigedennY-Konstruktion"" erfordert die Einstufung von Sehen und Bedienen auf einer 'pragmatischen Skala', was dazu führt, dass die Aussage Die meisten von ihnen haben zuvor noch nie einen Computer bedient als 'größere' Behauptung betrachtet werden muss als Die meisten von ihnen haben zuvor noch nie einen Computer gesehen. Mit anderen Worten: Wenn die meisten von ihnen einen Computer zuvor noch nie gesehen haben, dann haben sie ihn ganz sicher auch nicht bedient bzw. wenn sie einen Computer noch nie gesehen haben, können sie diesen auch nicht bedient haben. Proposition a) schließt Proposition b) somit aus. Unter dem Aspekt der Negation wird die erste Proposition Die meisten von ihnen haben zuvor noch nie einen Computer gesehen damit zur stärkeren der beiden kontrastierten Propositionen.","['»Ohne Europa sind viele Fragen nicht mehr seriös in Angriff zu nehmen , geschweige denn zu lösen« , meint der grüne Bundesparteisekretär Lothar Lockl .', 'Bis heute sind viele dieser Untaten , dieser Massaker an der »Heimatfront« , kaum aufgeklärt , geschweige denn im öffentlichen Gedächtnis präsent .', 'Plötzlich konnte Quelle keine Rechnung mehr bezahlen , geschweige denn den Hauptkatalog für den Herbst und Winter finanzieren .']","[""Ich habe heute eine Socke verloren , geschweige denn den ganzen Tag"", ""Das Auto, das ich gekauft habe, war zu teuer , geschweige denn zu ungerechtfertigt"", ""Ich habe gestern viel gegessen , geschweige denn zu viel""]",0.024875,0.046342,0.494391,0.522728,0.006056,0.005041
100,"Die ""Äquativ_Plural-Konstruktion"" gehört zu den Vergleichskonstruktionen und zeigt die Gleichheit zweier Entitäten an, die hinsichtlich eines bestimmten Wertes eines spezifischen Attributs auf einer Skala miteinander verglichen werden. Der 

In [92]:
print("Levenshtein ratio:")
print(examples[4]["4_ex_gen_levenshtein"].mean())
print(examples[4]["4_ex_gen_levenshtein"].std())
print("-----")
print(examples[4]["4_def_gen_levenshtein"].mean())
print(examples[4]["4_def_gen_levenshtein"].std())
print("=====")
print("Jaro-Winkler similarity score:")
print(examples[4]["4_ex_gen_jaro_winkler"].mean())
print(examples[4]["4_ex_gen_jaro_winkler"].std())
print("-----")
print(examples[4]["4_def_gen_jaro_winkler"].mean())
print(examples[4]["4_def_gen_jaro_winkler"].std())
print("=====")
print("BLEU score:")
print(examples[4]["4_ex_gen_bleu"].mean())
print(examples[4]["4_ex_gen_bleu"].std())
print("-----")
print(examples[4]["4_def_gen_bleu"].mean())
print(examples[4]["4_def_gen_bleu"].std())
examples[4]

Levenshtein ratio:
0.03190687782678974
0.01492259518946007
-----
0.1408555667682497
0.07790412359102843
=====
Jaro-Winkler similarity score:
0.45233205587348757
0.0898420571678826
-----
0.5317466904772062
0.032186789019234247
=====
BLEU score:
0.012877489859857235
0.009029325108903996
-----
0.007695101040245497
0.0065866771410472156


,definition,examples,generated,4_ex_gen_levenshtein,4_def_gen_levenshtein,4_ex_gen_jaro_winkler,4_def_gen_jaro_winkler,4_ex_gen_bleu,4_def_gen_bleu
10,"Die ""Negation:NEG_XgeschweigedennY-Konstruktion"" korreliert zwei Propositionen, die jeweils einen Punkt auf einem pragmatisch definiertem Maßstab markieren. Die erste Proposition ist pragmatisch stärker als die zweite Proposition; so zieht, pragmatisch betrachtet, die Wahrheit der ersten Proposition automatisch die Wahrheit der zweiten Proposition nach sich. Einfacher ausgedrückt: Ist die erste Proposition wahr, so muss die zweite Proposition ebenfalls wahr sein. Die erste Proposition ist darüber hinaus auch informativer als die zweite Proposition, da die erstgenannte pragmatisch betrachtet die letztgenannte einschließt, während diese wiederum die diskursrelevantere der beiden Propositionen ist. Die Konstruktion umfasst das konstruktionsevozierende Element (KEE) ""geschweige_denn"", die internen Kern-Konstruktionselemente (Kern-KE) ""Erstes_Konjunkt"" und ""Zweites_Konjunkt"" sowie die externen Kern-KE ""Negator"" und ""Fokuskontext"". Das ""Erste_Konjunkt"" geht dem ""KEE"" voraus, auf welches das ""Zweite_Konjunkt"" folgt. Der informationsstrukturelle Fokus liegt auf dem ""Ersten_Konjunkt"", das die Basis für den entstehenden Kontrast zwischen den zwei Propositionen bildet. Das ""Erste_Konjunkt"" und das ""Zweite_Konjunkt"" bilden jeweils einen Teil der beiden Propositionen ab, wobei das ""Erste_Konjunkt"" meist zusätzlih vom ""Negator"" negiert wird. Der ""Fokuskontext"" indiziert in der Regel den Rest der Proposition. Werden beide Konjunkte im Wechsel mit dem ""Fokuskontext"" kombiniert, können die einander gegenübergestellten Propositionen vollständig wiederhergestellt (oder: vervollständigt) werden. Im Gegensatz zum ""Ersten_Konjunkt"" und ""Zweiten_Konjunkt"" muss der ""Fokuskontext"" nicht zwangsläufig realisiert werden – wird der ""Fokuskontext"" nicht realisiert, drücken die beiden Konjunkte jeweils vollständige Propositionen aus. "" Die zwei gegenübergestellten Propositionen sind hier die beiden Aussagen, dass die meisten von ihnen zuvor noch nie einen Computer a) gesehen und b) bedient haben. Die Verwendung der ""Negation:NEG_XgeschweigedennY-Konstruktion"" erfordert die Einstufung von Sehen und Bedienen auf einer 'pragmatischen Skala', was dazu führt, dass die Aussage Die meisten von ihnen haben zuvor noch nie einen Computer bedient als 'größere' Behauptung betrachtet werden muss als Die meisten von ihnen haben zuvor noch nie einen Computer gesehen. Mit anderen Worten: Wenn die meisten von ihnen einen Computer zuvor noch nie gesehen haben, dann haben sie ihn ganz sicher auch nicht bedient bzw. wenn sie einen Computer noch nie gesehen haben, können sie diesen auch nicht bedient haben. Proposition a) schließt Proposition b) somit aus. Unter dem Aspekt der Negation wird die erste Proposition Die meisten von ihnen haben zuvor noch nie einen Computer gesehen damit zur stärkeren der beiden kontrastierten Propositionen.","['Ohne seine suggestiv - erhabenen Orchesterkompositionen hätten Filme wie Star Wars , Indiana Jones und E . T . wohl kaum jene sensationelle Wirkung entfaltet , dank der sich nicht nur das gesamte Hollywood - System grundlegend veränderte – sondern auch der orchestral eingespielte , sinfonische Soundtrack wieder zur beliebtesten Form der Filmmusik wurde .', 'Jeder Rest dumm - fröhlicher Ostalgie wird hier weggeätzt .', 'Auch er steht einem schwarz - roten Bündnis kritisch gegenüber :', 'Diese Extras verleihen den bittersüß - melancholischen Balladen und schmissigen Rockernummern , zwischen denen das Album pendelt , mehr Tiefe .']","[""Es ist schwer, die Trennung zwischen dem harmonischen und melodischen Melodien zu machen."", ""Der Regen rutschte auf den hellen Asphalt herunter und verweht die einladenden Schatten.""]",0.019157,0.059155,0.523316,0.516992,0.015100,0.012575
100,"Die ""Äquativ_Plural-Konstruktion"" gehört zu den Vergleichskonstruktionen 

In [95]:
for i, example in examples.items():
    example.to_excel(f"{i}_shot.xlsx")